Proyek Analisis

In [3]:
!pip install folium geopy

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from geopy.geocoders import Nominatim
import time
from pandas import ExcelFile
import ipywidgets as widgets
from IPython.display import display

Data Wrangling

Gathering Data

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
sales_df = pd.read_csv('/content/drive/MyDrive/dasetssim/carSales.csv', delimiter=';')
sales_df.head()

,Car_id,Date,Customer_Name,Gender,Annual_Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price,Dealer_No,Body_Style,Phone,Dealer_Region
0,C_CND_000001,01/02/2022,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,Double Overhead Camshaft,Auto,Black,26000,06457-3834,SUV,8264678,Middletown
1,C_CND_000002,01/02/2022,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,Double Overhead Camshaft,Auto,Black,19000,60504-7114,SUV,6848189,Aurora
2,C_CND_000003,01/02/2022,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,38701-8047,Passenger,7298798,Greenville
3,C_CND_000004,01/02/2022,Giselle,Male,13500,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000,99301-3882,SUV,6257557,Pasco
4,C_CND_000005,01/02/2022,Grace,Male,1465000,Chrysler Plymouth,Acura,TL,Double Overhead Camshaft,Auto,Red,24500,53546-9427,Hatchback,7081483,Janesville


Asessing Data

Menilai Tabel sales_df

In [7]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Car_id         23906 non-null  object
 1   Date           23906 non-null  object
 2   Customer_Name  23905 non-null  object
 3   Gender         23906 non-null  object
 4   Annual_Income  23906 non-null  int64 
 5   Dealer_Name    23906 non-null  object
 6   Company        23906 non-null  object
 7   Model          23906 non-null  object
 8   Engine         23906 non-null  object
 9   Transmission   23906 non-null  object
 10  Color          23906 non-null  object
 11  Price          23906 non-null  int64 
 12  Dealer_No      23906 non-null  object
 13  Body_Style     23906 non-null  object
 14  Phone          23906 non-null  int64 
 15  Dealer_Region  23906 non-null  object
dtypes: int64(3), object(13)
memory usage: 2.9+ MB


In [8]:
sales_df.isna().sum()

,0
Car_id,0
Date,0
Customer_Name,1
Gender,0
Annual_Income,0
Dealer_Name,0
Company,0
Model,0
Engine,0
Transmission,0


Cleaning Data

In [9]:
sales_df[sales_df.Customer_Name.isna()]

,Car_id,Date,Customer_Name,Gender,Annual_Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price,Dealer_No,Body_Style,Phone,Dealer_Region
7564,C_CND_007565,11/05/2022,NaN,Male,680000,Saab-Belle Dodge,Dodge,Ram Pickup,Double Overhead Camshaft,Auto,Pale White,45000,60504-7114,Hardtop,7203103,Aurora


In [10]:
sales_df.fillna(value="John Doe", inplace=True)

In [11]:
sales_df.isna().sum()

,0
Car_id,0
Date,0
Customer_Name,0
Gender,0
Annual_Income,0
Dealer_Name,0
Company,0
Model,0
Engine,0
Transmission,0


In [12]:
sales_df.Customer_Name.value_counts()

,count
Customer_Name,
Thomas,92
Emma,90
Lucas,88
Nathan,80
Louis,76
...,...
Bouzeau Flavie,1
Luigi,1
Maxent,1


## EDA

1. Data ini berasal dari negara mana?

In [13]:
# Ambil daftar unik region dealer
regions = sales_df['Dealer_Region'].dropna().unique()

# Inisialisasi geolocator
geolocator = Nominatim(user_agent="dealer_locator")

# Simpan hasil lokasi
locations = []

# Proses geocoding
for region in regions:
    try:
        location = geolocator.geocode(region + ", USA")
        if location:
            locations.append({
                'Region': region,
                'Latitude': location.latitude,
                'Longitude': location.longitude
            })
        time.sleep(1)
    except:
        continue

# Konversi ke DataFrame
locations_df = pd.DataFrame(locations)

# Tampilkan tabel
print(locations_df)

       Region   Latitude   Longitude
0  Middletown  41.562318  -72.650906
1      Aurora  41.757170  -88.314754
2  Greenville  34.851354  -82.398488
3       Pasco  28.299618  -82.452270
4  Janesville  42.682977  -89.022679
5  Scottsdale  33.494219 -111.926018
6      Austin  30.271129  -97.743700


2. Brand apa yang menjadi pesaing terberat di negara ini berdasarkan penjualan?

In [19]:
# Hitung jumlah pembelian per perusahaan
company_counts = sales_df['Company'].value_counts()

# Ambil 3 perusahaan teratas
top3_companies = company_counts.nlargest(3)

# Buat DataFrame
top3_df = top3_companies.reset_index()
top3_df.columns = ['Company', 'Total Sales']

# Hitung persentase kontribusi dari total penjualan top 3
total_top3 = top3_df['Total Sales'].sum()
top3_df['Percentage'] = (top3_df['Total Sales'] / total_top3 * 100).round(1)

# Tampilkan tabel
print(top3_df)


     Company  Total Sales  Percentage
0  Chevrolet         1819        35.6
1      Dodge         1671        32.7
2       Ford         1614        31.6


3. Apa 5 brand terbesar yang menguasai pasar di masing masing wilayah?

In [20]:
# Ambil 5 brand teratas berdasarkan jumlah penjualan
top5_brands = sales_df['Company'].value_counts().nlargest(5).index.tolist()

# Filter hanya baris yang termasuk 5 brand tersebut
top5_df = sales_df[sales_df['Company'].isin(top5_brands)]

# Buat tabel distribusi penjualan brand per region
table = pd.crosstab(top5_df['Dealer_Region'], top5_df['Company'])

# Urutkan berdasarkan total baris
table = table.loc[table.sum(axis=1).sort_values(ascending=False).index]

# Tampilkan tabel
print(table)


Company        Chevrolet  Dodge  Ford  Mercedes-B  Volkswagen
Dealer_Region                                                
Austin               324    305   274         208         227
Janesville           308    246   269         200         219
Scottsdale           243    223   235         176         203
Middletown           235    248   193         179         167
Greenville           229    226   208         168         185
Pasco                257    209   222         163         163
Aurora               223    214   213         191         169


4. Apa tipe gaya body yang paling banyak terjual di masing masing wilayah?

In [22]:
import pandas as pd

# Hitung jumlah penjualan per kombinasi region dan body style
region_body_counts = sales_df.groupby(['Dealer_Region', 'Body_Style']).size().reset_index(name='Count')

# Ambil 3 body style teratas untuk setiap region
top3_per_region = (
    region_body_counts
    .sort_values(['Dealer_Region', 'Count'], ascending=[True, False])
    .groupby('Dealer_Region')
    .head(3)
)

# Tampilkan tabel hasil
print(top3_per_region.sort_values(['Dealer_Region', 'Count'], ascending=[True, False]))


   Dealer_Region Body_Style  Count
3         Aurora        SUV    813
1         Aurora  Hatchback    803
4         Aurora      Sedan    587
8         Austin        SUV   1079
6         Austin  Hatchback   1024
9         Austin      Sedan    708
13    Greenville        SUV    878
11    Greenville  Hatchback    788
14    Greenville      Sedan    547
16    Janesville  Hatchback    987
18    Janesville        SUV    983
19    Janesville      Sedan    746
21    Middletown  Hatchback    822
23    Middletown        SUV    809
24    Middletown      Sedan    601
28         Pasco        SUV    824
26         Pasco  Hatchback    816
29         Pasco      Sedan    597
33    Scottsdale        SUV    988
31    Scottsdale  Hatchback    888
34    Scottsdale      Sedan    702


5. Berapa harga rata - rata setiap perusahaan di masing masing wilayah?

In [24]:
# Ambil 5 brand dengan penjualan terbanyak
top5_brands = sales_df['Company'].value_counts().nlargest(5).index.tolist()

# Filter data hanya untuk top 5 brand
top5_df = sales_df[sales_df['Company'].isin(top5_brands)]

# Hitung rata-rata harga per brand dan region
avg_price = top5_df.groupby(['Dealer_Region', 'Company'])['Price'].mean().reset_index()

# Bulatkan harga agar lebih rapi
avg_price['Price'] = avg_price['Price'].round(2)

# Tampilkan tabel
print(avg_price.sort_values(['Dealer_Region', 'Company']))


   Dealer_Region     Company     Price
0         Aurora   Chevrolet  27666.40
1         Aurora       Dodge  26185.82
2         Aurora        Ford  31439.68
3         Aurora  Mercedes-B  25100.98
4         Aurora  Volkswagen  25121.15
5         Austin   Chevrolet  25882.60
6         Austin       Dodge  26323.21
7         Austin        Ford  28809.17
8         Austin  Mercedes-B  26869.56
9         Austin  Volkswagen  26382.77
10    Greenville   Chevrolet  27618.37
11    Greenville       Dodge  26201.59
12    Greenville        Ford  28704.76
13    Greenville  Mercedes-B  27012.74
14    Greenville  Volkswagen  26307.81
15    Janesville   Chevrolet  26190.74
16    Janesville       Dodge  26545.32
17    Janesville        Ford  28661.22
18    Janesville  Mercedes-B  27711.80
19    Janesville  Volkswagen  26089.31
20    Middletown   Chevrolet  25216.64
21    Middletown       Dodge  26920.60
22    Middletown        Ford  28678.31
23    Middletown  Mercedes-B  27660.55
24    Middletown  Volkswa

6. Bagaimana tren penjualan mobil di wilayah masing masing setiap tahun?

In [25]:
import pandas as pd

# Pastikan kolom Date sudah dalam format datetime
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Buat kolom Bulan dan Tahun
sales_df['Month'] = sales_df['Date'].dt.to_period('M').astype(str)
sales_df['Year'] = sales_df['Date'].dt.year

# Filter hanya data dari tahun 2022 ke atas
sales_df = sales_df[sales_df['Year'] >= 2022]

# Hitung jumlah penjualan per bulan dan region
region_trend = sales_df.groupby(['Year', 'Month', 'Dealer_Region']).size().reset_index(name='Total_Sales')

# Ubah kolom Month jadi datetime agar bisa diurutkan
region_trend['Month'] = pd.to_datetime(region_trend['Month'])

# Urutkan berdasarkan waktu
region_trend = region_trend.sort_values(['Year', 'Month'])

# Tampilkan sebagai tabel pivot
trend_table = region_trend.pivot_table(
    index='Month',
    columns='Dealer_Region',
    values='Total_Sales',
    fill_value=0
)

# Tampilkan tabel
print(trend_table)


Dealer_Region  Aurora  Austin  Greenville  Janesville  Middletown  Pasco  \
Month                                                                      
2022-01-01       42.0    51.0        40.0        49.0        45.0   44.0   
2022-02-01       43.0    57.0        44.0        48.0        41.0   42.0   
2022-03-01       92.0   117.0        96.0       113.0        94.0   90.0   
2022-04-01      105.0   140.0       110.0       121.0       104.0  103.0   
2022-05-01       96.0   141.0        93.0       119.0       100.0   96.0   
2022-06-01       83.0   121.0        83.0       115.0        91.0   93.0   
2022-07-01       92.0   127.0        91.0       111.0        93.0   86.0   
2022-08-01      103.0   136.0       104.0       137.0       113.0  101.0   
2022-09-01      187.0   249.0       188.0       250.0       184.0  191.0   
2022-10-01      112.0   143.0       114.0       135.0       114.0  105.0   
2022-11-01      218.0   296.0       210.0       254.0       216.0  209.0   
2022-12-01  